# Fuzzy String Matching and Crew Donation Stats program

This is a Python notebook (basically a collection of cells containing various code blocks), intended for fuzzy string matching of names in Fantasia Staff Crew sheet and OCR Sheet from in-game crew page screenshot.

Author: Ani

Set the parameters below in Step 1, and then you can go to Runtime -> Run all, and follow the other steps of uploading the two files as instructed below. You might need to wait a few seconds before the Choose Files prompt becomes available.

## Step 1: Setting the Parameters

Except Fantasia, default parameter values here can mostly work fine, but here they can be customized as needed.

- `Username Column Name`: This is the column name of the username field in the Staff Spreadsheet. This is by default `ToF Username` for Phantasia and Sacred Sakura, but `ToF Username - {number of members}`. For Fantasia, this column name needs to be set here.

- `Output File Name`: This is the filename of the output csv file, by default set to `CrewDonationStats.csv`. Can leave this as is, or change to anything else that you wish. Just be sure to have the `.csv` extension at the end.

- `Score Cutoff`: The Fuzzy String matching program code here computes a similarity score between 0 and 100. Two exactly same names will have a score of 100. Here we set a score cutoff for finding matches between names in the Crew sheet and the Staff sheet. The default is 70, but this can be increased or decreased if you want to experiment.

- `Mode`: There are two modes. `staff-to-crew` mode matches staff spreadsheet names to crew screenshot names. This is the default, and should be used to fill up weekly donation info. Blank entries here will mean that the name in spreadsheet wasn't found in crew screenshot. This could be due to very short name, incorrect OCR reading from crew screenshot, or someone in crew changing their username which wasn't tracked in staff spreadsheet. For the case that sometimes players change their username, there exists the `crew-to-staff` mode, which matches crew screenshot names to staff spreadsheet names. Blank spaces should help to find names in spreadsheet which don't exist in crew screenshot. (Of course, it could be that short names or incorrect OCR reading messed up, but this simplifies finding out who changed their name).

Once these are set, you can now go to **Runtime -> Run all**. Make sure to upload the two files below as instructed, one by one.

**You should still check the final output in the output file manually once. Short names, and blank names can be easily missed, so fill in any missing details yourself by cross-checking with the Crew page screenshot if applicable.**

In [ ]:
# @title Relevant Parameters {display-mode: "form"}

username_column_name = "ToF Username"  # @param {type: "string"}
output_filename = "CrewDonationStats.csv"  # @param {type: "string"}
score_cutoff = 70  # @param {type: "slider", min: 0, max: 100}
mode = "staff-to-crew" # @param ["staff-to-crew", "crew-to-staff"]



**Go to Runtime -> Run All and follow the instructions you see below in the notebook.**

## Step 2: Upload the CSV file from Staff Spreadsheet.

In [ ]:
# @title Upload the CSV file from Staff Spreadsheet. See instructions below. {display-mode: "form"}

from google.colab import files

print(f"1. Open the Fantasia Staff Spreadsheet.")
print(f"2. Go to your crew sheet (Fantasia/Phantasia/Sacred Sakura).")
print(f"3. Right click on File Menu -> Download -> Comma-separated values (.csv)")
print(f"4. Click on the Choose Files button below, select the above downloaded file from your computer, and upload it here.")

staffsheet = files.upload()

for fn in staffsheet.keys():
    print(f"Uploaded file '{fn}'")

staffsheetname = list(staffsheet.keys())[0]

In [ ]:
# @title Step 3. Upload the CSV file from TableReader website (Long Image from Crew Screen in-game converted to Table and exported as .csv) {display-mode: "form"}

print(f"Upload the CSV file from TableReader website. Please only upload one file here.")
crewsheet = files.upload()

for fn in crewsheet.keys():
    print(f"Uploaded file '{fn}'")

crewsheetname = list(crewsheet.keys())[0]

In [ ]:
# @title Working code {display-mode: "form"}

!pip install thefuzz --quiet

import os
import pandas as pd
from thefuzz import fuzz, process

# Load the CSV File
crewdf = pd.read_csv(crewsheetname,
                     header=None,
                     names=['SrNo.', 'Name', 'Level', 'Rank', 'WeeklyDonation', 'HistoricalDonation', 'LastActive']
                    )

# Remove the Sr No. column which doesn't have any use
crewdf.drop(columns=['SrNo.'], inplace=True)

# Fill missing weekly and historical donation values with 0
crewdf['WeeklyDonation'].fillna(0, inplace=True)
crewdf['HistoricalDonation'].fillna(0, inplace=True)

# Replace "O" with 0
crewdf['WeeklyDonation'].replace(regex=r"\s*O\s*", value="0", inplace=True)

# Remove whitespaces from the name
crewdf['Name'] = crewdf['Name'].apply(lambda x: str(x).strip())

# Load Staff Sheet
sheetdf = pd.read_csv(staffsheetname, skiprows=1)

sheetnames = list(sheetdf[username_column_name].dropna())

crewdonationstats = []

if mode == 'crew-to-staff':
    for name in crewdf['Name']:
        result = process.extractOne(name, sheetnames, score_cutoff=score_cutoff, scorer=fuzz.ratio)

        if result:
            sheetname, score = result
        else:
            sheetname = ""

        statsdict = {
                'CrewOCRName': name,
                'SheetName': sheetname
            }

        crewdonationstats.append(statsdict)

    crewdonationstatsdf = pd.DataFrame(crewdonationstats)

    crewdonationstatsdf.to_csv(output_filename, index=False)

elif mode == "staff-to-crew":
    for name in sheetnames:
        result = process.extractOne(name, crewdf['Name'], score_cutoff=score_cutoff, scorer=fuzz.ratio)

        if result:
            crewocrname, score, key = result
            weeklydonation = crewdf.iloc[key]['WeeklyDonation']
            historicaldonation = crewdf.iloc[key]['HistoricalDonation']
        else:
            crewocrname = ""
            weeklydonation = ""
            historicaldonation = ""

        statsdict = {
                'SheetName': name,
                'CrewOCRName': crewocrname,
                'WeeklyDonation': weeklydonation,
                'HistoricalDonation': historicaldonation
            }

        crewdonationstats.append(statsdict)

    crewdonationstatsdf = pd.DataFrame(crewdonationstats)

    crewdonationstatsdf.to_csv(output_filename, index=False)

else:
    raise Exception("You probably messed up the mode setting somehow. Close this notebook, and launch a fresh instance. Contact Ani if issue persists.")

# crewdonationstatsdf

In [ ]:
# @title Download file {display-mode: "form"}

files.view(output_filename)
files.download(output_filename)

In [ ]:
assert False, "This cell will error out. Don't worry. It's so that the cell below can be run manually, if need."

In [ ]:
# @title Click on this cell, and run it (The play icon on the left, or `CTRL+ENTER` as a keyboard shortcut). This will disconnect the runtime. {display-mode: "form"}
# https://twitter.com/GoogleColab/status/1569389018311426051


from google.colab import runtime

runtime.unassign()

## Step 4: Putting the data back in the sheet

A CSV file should be opened on the right side, and the same also downloaded on your computer. The view on the right shows the output from the program.

You can open the download file on your computer, and copy paste the relevant columns into your own staff sheet as necessary.